In [49]:
#Importar libraries
import pandas as pd

In [58]:
def process_dicionario(df, column= 'PALAVRA'):
    df[column] = df[column].str.strip() #remove espaços em branco
    df[column] = df[column].str.lower() #transforma em minúsculo
    df[column] = df[column].str.replace(' ', '_') #substitui espaço por underline
    df[column] = df[column].str.replace('\W', '', regex=True) #remove caracteres especiais
    df[column] = df[column].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') #remove acentos
    return df

In [59]:
#importar dicionarios
dicionario_ambiental = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='AMBIENTAL')
dicionario_social = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='SOCIAL')
dicionario_governanca = pd.read_excel('V2. DICIONÁRIO COMPILADOS ING.POR.xlsx',sheet_name='GOVERNANÇA')

In [63]:
dicionario_ambiental = process_dicionario(dicionario_ambiental)
dicionario_social = process_dicionario(dicionario_social)
dicionario_governanca = process_dicionario(dicionario_governanca)

In [64]:
dicionario_ambiental['PALAVRA']

0       mudancas_climaticas
1            titulos_verdes
2        combustivel_fossil
3              titulo_verde
4        emissao_de_carbono
               ...         
142    gas_de_efeito_estuda
143      substancia_quimica
144          emissao_de_gas
145     seguranca_ambiental
146      consumo_de_energia
Name: PALAVRA, Length: 147, dtype: object

In [67]:
dicionario_social['PALAVRA']

0                                dinheiro_moral
1                      investimento_responsavel
2                  objetivos_de_desenvolvimento
3      objetivos_de_desenvolvimento_sustentavel
4                       investimento_de_impacto
                         ...                   
218                     programa_de_treinamento
219                    responsabilidade_soacial
220                            recursos_humanos
221                       consumidor_financeiro
222                                novo_negocio
Name: PALAVRA, Length: 223, dtype: object

In [74]:
dic_governanca = list(dicionario_governanca['PALAVRA'])
dic_governanca

['fundos_de_pensao',
 'gestao_de_investimentos',
 'cadeia_de_suprimentos',
 'forca_tarefa',
 'gestores_de_investimento',
 'diretor_de_investimentos',
 'questoes_de_governanca',
 'setor_privado',
 'fundos_de_hedge',
 'diretor_administrativo',
 'propostas_dos_acionistas',
 'diligencia_devida',
 'capitalismo_das_partes_interessadas',
 'investidores_de_varejo',
 'reunioes_anuais',
 'divulgacao_esg',
 'escritorio_de_advocacia',
 'conselheiros_globais',
 'membros_do_conselho',
 'investidores_procurando',
 'gerentes_passivos',
 'investidores_institucionais',
 'conselheiros',
 'recompensa',
 'kyc',
 'lista_branca',
 'blockchain',
 'utilitario',
 'token_de_seguranca',
 'distribuicao_de_token',
 'intermediario',
 'lei',
 'regulacao',
 'politica',
 'regulador',
 'retencao_de_token',
 'lancamento_aereo',
 'fundados',
 'parceiro',
 'conformidade',
 'teste_de_howey',
 'cvm',
 'equidade',
 'capital_de_risco',
 'cr',
 'incubador',
 'alinhar',
 'alinhado',
 'alinhando',
 'alinhamento',
 'alinha',
 'est

In [77]:
# import dados dos hoteis
dados = pd.read_excel('./COMENTS TRIP. HOTÉIS. NOMES CORRETOS. SEM COMENT REPETIDO/ACasaRosa Hostel.xlsx')

In [81]:
#importa os dados primarios
dados_primarios =   pd.read_excel('DadosLuisaFinalV4.xlsx')

In [83]:
dados_primarios['Nome do Hotel']

0                      ACasaRosa Hostel
1                 Apoema Suites e Flats
2                           Apple House
3                     Bahia Inn Pousada
4                            Casa Clara
                     ...               
191              VILLA CANZIANI &DONATO
192              VILLA ECOPORAN ITACARE
193    Vitória Hotel  Express Dom Pedro
194              Windsor Brasilia Hotel
195        Windsor Plaza Brasilia Hotel
Name: Nome do Hotel, Length: 196, dtype: object